### Testing on new unseen data

In [52]:
from openai import OpenAI
from dotenv import load_dotenv
import pandas as pd

In [53]:
load_dotenv()

True

In [54]:
client = OpenAI()

In [56]:
FINETUNED_MODEL = "ft:gpt-4o-mini-2024-07-18:vs::B43KoeA3"
BASE_MODEL = "gpt-4o-mini-2024-07-18"

In [57]:
system_message = """Extract the medical report information into the following model:
{'$defs': {'Action': {'enum': ['HYDRATION', 'STRESS_MANAGEMENT', 'SLEEP', 'EXERCISE', 'DIETARY_CHANGES', 'MEDICATION_ADHERENCE', 'REGULAR_CHECKUPS', 'AVOID_ALCOHOL', 'SMOKING_CESSATION', 'BALANCED_DIET', 'PHYSICAL_THERAPY', 'MENTAL_HEALTH_SUPPORT', 'PAIN_MANAGEMENT', 'IMMUNIZATION', 'FOLLOW_UP_VISITS', 'WEIGHT_MANAGEMENT', 'BLOOD_PRESSURE_MONITORING', 'BLOOD_SUGAR_MONITORING', 'CHOLESTEROL_CONTROL', 'FLUID_INTAKE_MONITORING'], 'title': 'Action', 'type': 'string'}, 'Address': {'properties': {'street': {'title': 'Street', 'type': 'string'}, 'city': {'title': 'City', 'type': 'string'}, 'state': {'title': 'State', 'type': 'string'}, 'zip_code': {'title': 'Zip Code', 'type': 'string'}}, 'required': ['street', 'city', 'state', 'zip_code'], 'title': 'Address', 'type': 'object'}, 'Diagnosis': {'enum': ['FLU', 'COLD', 'COVID19', 'ASTHMA', 'DIABETES', 'HYPERTENSION', 'MIGRAINE', 'ANEMIA', 'ARTHRITIS', 'DEPRESSION', 'ALLERGY', 'BRONCHITIS'], 'title': 'Diagnosis', 'type': 'string'}, 'Medication': {'enum': ['PARACETAMOL', 'IBUPROFEN', 'AMOXICILLIN', 'METFORMIN', 'ALBUTEROL', 'LISINOPRIL', 'ATORVASTATIN', 'ASPIRIN', 'OMEPRAZOLE', 'LEVOTHYROXINE', 'SIMVASTATIN', 'LOSARTAN'], 'title': 'Medication', 'type': 'string'}, 'Patient': {'properties': {'name': {'description': 'The first name of the patient', 'title': 'Name', 'type': 'string'}, 'surname': {'description': 'The last name of the patient', 'title': 'Surname', 'type': 'string'}, 'age': {'description': 'The age of the patient, if not specified = 0', 'title': 'Age', 'type': 'integer'}, 'gender': {'description': 'The gender of the patient - male or female', 'title': 'Gender', 'type': 'string'}, 'address': {'$ref': '#/$defs/Address', 'description': 'The address of the patient'}, 'phone_number': {'description': 'The phone number of the patient', 'title': 'Phone Number', 'type': 'string'}, 'email': {'description': 'The email of the patient', 'title': 'Email', 'type': 'string'}}, 'required': ['name', 'surname', 'age', 'gender', 'address', 'phone_number', 'email'], 'title': 'Patient', 'type': 'object'}, 'Symptom': {'enum': ['FEVER', 'COUGH', 'FATIGUE', 'SHORTNESS_OF_BREATH', 'NAUSEA', 'HEADACHE', 'DIZZINESS', 'SORE_THROAT', 'RUNNY_NOSE', 'MUSCLE_ACHES', 'LOSS_OF_TASTE', 'LOSS_OF_SMELL', 'CHEST_PAIN', 'CHILLS'], 'title': 'Symptom', 'type': 'string'}, 'TreatmentPlan': {'properties': {'time_period': {'description': 'The time period for the treatment plan in days, if not specified = 1', 'title': 'Time Period', 'type': 'integer'}, 'actions': {'items': {'$ref': '#/$defs/Action'}, 'title': 'Actions', 'type': 'array'}, 'medications': {'items': {'$ref': '#/$defs/Medication'}, 'title': 'Medications', 'type': 'array'}}, 'required': ['time_period', 'actions', 'medications'], 'title': 'TreatmentPlan', 'type': 'object'}}, 'properties': {'patient': {'$ref': '#/$defs/Patient'}, 'diagnosis': {'$ref': '#/$defs/Diagnosis'}, 'treatment_plan': {'$ref': '#/$defs/TreatmentPlan'}, 'symptoms': {'items': {'$ref': '#/$defs/Symptom'}, 'title': 'Symptoms', 'type': 'array'}}, 'required': ['patient', 'diagnosis', 'treatment_plan', 'symptoms'], 'title': 'MedicalReport', 'type': 'object'}

If something is not clear, or incomplete, leave it blank."""

In [58]:
completion = client.chat.completions.create(
    model=FINETUNED_MODEL,
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": "Samantha Taylor, a 45-year-old woman residing at 78 Willow Creek Drive, Hartford, has been experiencing persistent symptoms for the past week. She noticed an overwhelming sense of fatigue that seems to worsen despite rest, coupled with frequent bouts of dizziness and headaches. Samantha also reported occasional nausea. Concerned about her deteriorating health, she visited her healthcare provider. Upon examination, she was diagnosed with a type of flu. Her doctor, Dr. Mercer, prescribed a treatment plan involving a seven-day course of rest, hydration, and regular administration of Paracetamol and Ibuprofen to manage her symptoms. They also advised her to monitor her symptoms closely and report any changes. Samantha, who can be reached via her cellphone at 987654321 and through her email at samantha.taylor@example.com, is determined to follow the treatment plan closely to ensure a swift recovery."}
    ]
)

response = completion.choices[0].message

print(response.content)

{"patient": {"name": "Samantha", "surname": "Taylor", "age": 45, "gender": "female", "address": {"street": "78 Willow Creek Drive", "city": "Hartford", "state": "", "zip_code": ""}, "phone_number": "987654321", "email": "samantha.taylor@example.com"}, "diagnosis": "FLU", "treatment_plan": {"time_period": 7, "actions": ["HYDRATION", "SLEEP"], "medications": ["PARACETAMOL", "IBUPROFEN"]}, "symptoms": ["FATIGUE", "DIZZINESS", "HEADACHE", "NAUSEA"]}


In [59]:
test = pd.read_csv("datasets/test.csv")

In [60]:
test.head()

,prompt,completion,system
0,"After a few days of feeling under the weather,...","{""patient"": {""name"": ""George"", ""surname"": ""Sum...",Extract the medical report information into th...
1,"Despite the tropical heat enveloping Miami, 72...","{""patient"": {""name"": ""Maria"", ""surname"": ""Alva...",Extract the medical report information into th...
2,"Samantha Taylor, a 45-year-old woman residing ...","{""patient"": {""name"": ""Samantha"", ""surname"": ""T...",Extract the medical report information into th...
3,"Claude Walker, a 47-year-old male from 7th Ave...","{""patient"": {""name"": ""Claude"", ""surname"": ""Wal...",Extract the medical report information into th...
4,"Luis Ortega, a 52-year-old male, has been feel...","{""patient"": {""name"": ""Luis"", ""surname"": ""Orteg...",Extract the medical report information into th...


In [61]:
def get_prediction(prompt, model):
    
    completion = client.chat.completions.create(
        model=model,
        messages = [
            {"role": "system", "content": system_message},
            {"role": "user", "content": prompt}
        ]
    )

    response = completion.choices[0].message

    print(response.content)

    return response.content

In [62]:
test['predicted_output'] = test['prompt'].apply(lambda prompt: get_prediction(prompt, model=FINETUNED_MODEL))

{"patient": {"name": "George", "surname": "Summers", "age": 42, "gender": "male", "address": {"street": "789 Palm Avenue", "city": "Clearwater Beach", "state": "", "zip_code": ""}, "phone_number": "(813) 987-6543", "email": "george.summers@beachmail.com"}, "diagnosis": "COLD", "treatment_plan": {"time_period": 7, "actions": ["HYDRATION", "SLEEP"], "medications": ["PARACETAMOL", "IBUPROFEN"]}, "symptoms": ["COUGH", "FATIGUE", "HEADACHE"]}
{"patient": {"name": "Maria", "surname": "Alvarez", "age": 72, "gender": "female", "address": {"street": "Coral Street", "city": "Miami", "state": "", "zip_code": ""}, "phone_number": "silver-lined flip phone", "email": ""}, "diagnosis": "FLU", "treatment_plan": {"time_period": 7, "actions": ["HYDRATION", "SLEEP"], "medications": ["PARACETAMOL", "IBUPROFEN"]}, "symptoms": ["FEVER", "HEADACHE", "COUGH", "FATIGUE"]}
{"patient": {"name": "Samantha", "surname": "Taylor", "age": 45, "gender": "female", "address": {"street": "78 Willow Creek Drive", "city": 

In [64]:
test.head()

,prompt,completion,system,predicted_output
0,"After a few days of feeling under the weather,...","{""patient"": {""name"": ""George"", ""surname"": ""Sum...",Extract the medical report information into th...,"{""patient"": {""name"": ""George"", ""surname"": ""Sum..."
1,"Despite the tropical heat enveloping Miami, 72...","{""patient"": {""name"": ""Maria"", ""surname"": ""Alva...",Extract the medical report information into th...,"{""patient"": {""name"": ""Maria"", ""surname"": ""Alva..."
2,"Samantha Taylor, a 45-year-old woman residing ...","{""patient"": {""name"": ""Samantha"", ""surname"": ""T...",Extract the medical report information into th...,"{""patient"": {""name"": ""Samantha"", ""surname"": ""T..."
3,"Claude Walker, a 47-year-old male from 7th Ave...","{""patient"": {""name"": ""Claude"", ""surname"": ""Wal...",Extract the medical report information into th...,"{""patient"": {""name"": ""Claude"", ""surname"": ""Wal..."
4,"Luis Ortega, a 52-year-old male, has been feel...","{""patient"": {""name"": ""Luis"", ""surname"": ""Orteg...",Extract the medical report information into th...,"{""patient"": {""name"": ""Luis"", ""surname"": ""Orteg..."


In [65]:
test.to_csv("datasets/test_predicted.csv", index=False)

In [66]:
test_predicted = pd.read_csv("datasets/test_predicted.csv")

In [68]:
test_predicted.head()

,prompt,completion,system,predicted_output
0,"After a few days of feeling under the weather,...","{""patient"": {""name"": ""George"", ""surname"": ""Sum...",Extract the medical report information into th...,"{""patient"": {""name"": ""George"", ""surname"": ""Sum..."
1,"Despite the tropical heat enveloping Miami, 72...","{""patient"": {""name"": ""Maria"", ""surname"": ""Alva...",Extract the medical report information into th...,"{""patient"": {""name"": ""Maria"", ""surname"": ""Alva..."
2,"Samantha Taylor, a 45-year-old woman residing ...","{""patient"": {""name"": ""Samantha"", ""surname"": ""T...",Extract the medical report information into th...,"{""patient"": {""name"": ""Samantha"", ""surname"": ""T..."
3,"Claude Walker, a 47-year-old male from 7th Ave...","{""patient"": {""name"": ""Claude"", ""surname"": ""Wal...",Extract the medical report information into th...,"{""patient"": {""name"": ""Claude"", ""surname"": ""Wal..."
4,"Luis Ortega, a 52-year-old male, has been feel...","{""patient"": {""name"": ""Luis"", ""surname"": ""Orteg...",Extract the medical report information into th...,"{""patient"": {""name"": ""Luis"", ""surname"": ""Orteg..."


In [ ]:
import json

# Convert JSON strings to dictionaries
def parse_json(json_str):
    try:
        return json.loads(json_str)
    except json.JSONDecodeError:
        return None

# Compare nested dictionaries and calculate similarity
def calculate_similarity(expected, predicted, path=""):
    if isinstance(expected, dict) and isinstance(predicted, dict):
        total_keys = len(expected)
        matched_keys = 0
        
        for key in expected:
            sub_path = f"{path}.{key}" if path else key
            if key in predicted:
                matched_keys += calculate_similarity(expected[key], predicted[key], sub_path)
        
        return matched_keys / total_keys if total_keys > 0 else 1
    
    elif isinstance(expected, list) and isinstance(predicted, list):
        # Compare lists as sets to ignore order
        matched_items = len(set(expected) & set(predicted))
        total_items = len(expected) if expected else 1
        return matched_items / total_items
    
    else:
        # Compare values directly
        return 1 if expected == predicted else 0




In [70]:
# Apply this to the test data
test_predicted['expected_dict'] = test_predicted['completion'].apply(parse_json)
test_predicted['predicted_dict'] = test_predicted['predicted_output'].apply(parse_json)

# Calculate similarity score for each row
test_predicted['similarity_score'] = test_predicted.apply(
    lambda row: calculate_similarity(row['expected_dict'], row['predicted_dict']),
    axis=1
)

# Convert to percentage
test_predicted['similarity_percentage'] = test_predicted['similarity_score'] * 100

# Calculate average similarity across all rows
average_similarity = test_predicted['similarity_percentage'].mean()
print(f"Average Similarity: {average_similarity:.2f}%")

Average Similarity: 98.01%


In [71]:
test_predicted['predicted_output_base_model'] = test_predicted['prompt'].apply(lambda prompt: get_prediction(prompt, model=BASE_MODEL))

{
  "patient": {
    "name": "George",
    "surname": "Summers",
    "age": 42,
    "gender": "male",
    "address": {
      "street": "789 Palm Avenue",
      "city": "Clearwater Beach",
      "state": "",
      "zip_code": ""
    },
    "phone_number": "(813) 987-6543",
    "email": "george.summers@beachmail.com"
  },
  "diagnosis": "COLD",
  "treatment_plan": {
    "time_period": 7,
    "actions": [
      "FLUID_INTAKE_MONITORING",
      "REST",
      "AVOID_STRENUOUS_ACTIVITIES"
    ],
    "medications": [
      "PARACETAMOL",
      "IBUPROFEN"
    ]
  },
  "symptoms": [
    "COUGH",
    "FATIGUE",
    "HEADACHE"
  ]
}
{
  "patient": {
    "name": "Maria",
    "surname": "Alvarez",
    "age": 72,
    "gender": "female",
    "address": {
      "street": "Coral Street",
      "city": "Miami",
      "state": "",
      "zip_code": ""
    },
    "phone_number": "Silver-lined flip phone with Isabella's number on speed dial",
    "email": ""
  },
  "diagnosis": "FLU",
  "treatment_plan": 

In [72]:
test_predicted.head()

,prompt,completion,system,predicted_output,expected_dict,predicted_dict,similarity_score,similarity_percentage,predicted_output_base_model
0,"After a few days of feeling under the weather,...","{""patient"": {""name"": ""George"", ""surname"": ""Sum...",Extract the medical report information into th...,"{""patient"": {""name"": ""George"", ""surname"": ""Sum...","{'patient': {'name': 'George', 'surname': 'Sum...","{'patient': {'name': 'George', 'surname': 'Sum...",1.000000,100.000000,"{\n ""patient"": {\n ""name"": ""George"",\n ..."
1,"Despite the tropical heat enveloping Miami, 72...","{""patient"": {""name"": ""Maria"", ""surname"": ""Alva...",Extract the medical report information into th...,"{""patient"": {""name"": ""Maria"", ""surname"": ""Alva...","{'patient': {'name': 'Maria', 'surname': 'Alva...","{'patient': {'name': 'Maria', 'surname': 'Alva...",0.905357,90.535714,"{\n ""patient"": {\n ""name"": ""Maria"",\n ""..."
2,"Samantha Taylor, a 45-year-old woman residing ...","{""patient"": {""name"": ""Samantha"", ""surname"": ""T...",Extract the medical report information into th...,"{""patient"": {""name"": ""Samantha"", ""surname"": ""T...","{'patient': {'name': 'Samantha', 'surname': 'T...","{'patient': {'name': 'Samantha', 'surname': 'T...",1.000000,100.000000,"{\n ""patient"": {\n ""name"": ""Samantha"",\n ..."
3,"Claude Walker, a 47-year-old male from 7th Ave...","{""patient"": {""name"": ""Claude"", ""surname"": ""Wal...",Extract the medical report information into th...,"{""patient"": {""name"": ""Claude"", ""surname"": ""Wal...","{'patient': {'name': 'Claude', 'surname': 'Wal...","{'patient': {'name': 'Claude', 'surname': 'Wal...",0.958333,95.833333,"```json\n{\n ""patient"": {\n ""name"": ""Claud..."
4,"Luis Ortega, a 52-year-old male, has been feel...","{""patient"": {""name"": ""Luis"", ""surname"": ""Orteg...",Extract the medical report information into th...,"{""patient"": {""name"": ""Luis"", ""surname"": ""Orteg...","{'patient': {'name': 'Luis', 'surname': 'Orteg...","{'patient': {'name': 'Luis', 'surname': 'Orteg...",1.000000,100.000000,"{\n ""patient"": {\n ""name"": ""Luis"",\n ""s..."


In [74]:
# Apply this to the test data
#test_predicted['expected_dict'] = test_predicted['completion'].apply(parse_json)
test_predicted['predicted_dict_base_model'] = test_predicted['predicted_output_base_model'].apply(parse_json)

# Calculate similarity score for each row
test_predicted['similarity_score_base_model'] = test_predicted.apply(
    lambda row: calculate_similarity(row['expected_dict'], row['predicted_dict_base_model']),
    axis=1
)

# Convert to percentage
test_predicted['similarity_percentage_base_model'] = test_predicted['similarity_score_base_model'] * 100

# Calculate average similarity across all rows
average_similarity = test_predicted['similarity_percentage_base_model'].mean()
print(f"Average Similarity (Base Model): {average_similarity:.2f}%")

Average Similarity (Base Model): 86.76%


In [75]:
test_predicted.head()

,prompt,completion,system,predicted_output,expected_dict,predicted_dict,similarity_score,similarity_percentage,predicted_output_base_model,predicted_dict_base_model,similarity_score_base_model,similarity_percentage_base_model
0,"After a few days of feeling under the weather,...","{""patient"": {""name"": ""George"", ""surname"": ""Sum...",Extract the medical report information into th...,"{""patient"": {""name"": ""George"", ""surname"": ""Sum...","{'patient': {'name': 'George', 'surname': 'Sum...","{'patient': {'name': 'George', 'surname': 'Sum...",1.000000,100.000000,"{\n ""patient"": {\n ""name"": ""George"",\n ...","{'patient': {'name': 'George', 'surname': 'Sum...",0.916667,91.666667
1,"Despite the tropical heat enveloping Miami, 72...","{""patient"": {""name"": ""Maria"", ""surname"": ""Alva...",Extract the medical report information into th...,"{""patient"": {""name"": ""Maria"", ""surname"": ""Alva...","{'patient': {'name': 'Maria', 'surname': 'Alva...","{'patient': {'name': 'Maria', 'surname': 'Alva...",0.905357,90.535714,"{\n ""patient"": {\n ""name"": ""Maria"",\n ""...","{'patient': {'name': 'Maria', 'surname': 'Alva...",0.780357,78.035714
2,"Samantha Taylor, a 45-year-old woman residing ...","{""patient"": {""name"": ""Samantha"", ""surname"": ""T...",Extract the medical report information into th...,"{""patient"": {""name"": ""Samantha"", ""surname"": ""T...","{'patient': {'name': 'Samantha', 'surname': 'T...","{'patient': {'name': 'Samantha', 'surname': 'T...",1.000000,100.000000,"{\n ""patient"": {\n ""name"": ""Samantha"",\n ...","{'patient': {'name': 'Samantha', 'surname': 'T...",1.000000,100.000000
3,"Claude Walker, a 47-year-old male from 7th Ave...","{""patient"": {""name"": ""Claude"", ""surname"": ""Wal...",Extract the medical report information into th...,"{""patient"": {""name"": ""Claude"", ""surname"": ""Wal...","{'patient': {'name': 'Claude', 'surname': 'Wal...","{'patient': {'name': 'Claude', 'surname': 'Wal...",0.958333,95.833333,"```json\n{\n ""patient"": {\n ""name"": ""Claud...",None,0.000000,0.000000
4,"Luis Ortega, a 52-year-old male, has been feel...","{""patient"": {""name"": ""Luis"", ""surname"": ""Orteg...",Extract the medical report information into th...,"{""patient"": {""name"": ""Luis"", ""surname"": ""Orteg...","{'patient': {'name': 'Luis', 'surname': 'Orteg...","{'patient': {'name': 'Luis', 'surname': 'Orteg...",1.000000,100.000000,"{\n ""patient"": {\n ""name"": ""Luis"",\n ""s...","{'patient': {'name': 'Luis', 'surname': 'Orteg...",1.000000,100.000000


In [76]:
test_predicted.to_csv("datasets/test_predicted.csv", index=False)

### Fine-Tuning Metrics
* Training loss: 0.0249
* Validation loss: 0.0900
* Full validation loss: 0.0262